In [1]:
import time
start_time = time.time()

import numpy as np
import json
import os
import matplotlib.pyplot as plt
import pickle
import sys
import astropy.units as u
import glob
import h5py

from msafit.fitting.likelihood import *
from msafit.fitting.fit_model import FitModel
from msafit.fitting.fit import prep_obs, run_fit_optimizer
from msafit.fitting.dependencies import *
from msafit.fitting import priors
from msafit.model import Sersic, ThinDisk
from msafit.fpa import PSFLib
#from msafit.plotting import plot_data
from astropy.table import Table
from functools import partial

from select_obs_mock import select_obs


os.environ["OMP_NUM_THREADS"] = "1"

#----------------------------------------------------------------------------#
#change to your paths 
# set paths to data 
datadir = "/u/flolac/Mocks/" #where cutout mocks are stored 
output_dir = "/u/flolac/"+"fit_results/"
figdir = "/u/flolac/Figures/"

# options for fitting
fix_position = True
fix_nsersic = True
fix_rt = True
store_fmodel = True
rnum = None

# set this to True to actually run the fit
optimize = False #Set to false to check if loading works then if True it runs the fitting 

#----------------------------------------------------------------------------#

f_norm = 1.0  # if needed, some scaling factor to avoid very small or large numbers and associated rounding errors
zred = 2.0 #redshift 
l0 = 6564.6 *(1+zred) # input wavelength same than the one used for creating the mock 


# auto-create output directories to store results
if rnum is None:
    ldir = glob.glob(output_dir+'/*')
    if len(ldir)==0:
        rnum = 1
    else:
        nums = []
        for dname in ldir:
            nums.append(int(dname.split('/')[-1][3:]))
        last_num = sorted(nums)[-1]
        rnum = last_num + 1
    wdir = output_dir + f'run{rnum}/'
    os.makedirs(wdir,exist_ok=True)
else:
    wdir = output_dir + f'run{rnum}/'
    os.makedirs(wdir,exist_ok=True)


# update these to reflect where your data live, and how they were constructed
fnames = ['mock_1.npy'] #file names for the mock data 
enames = ['error_array_1.npy'] # for the uncertainties 
shutters_i = [183]  #Whatever I used to make the mock in the first place 
shutters_j = [85] #""""
qd = 3 # quadrons check it's the righ quadrans
filt = 'F170LP' #Set the right filters 
disp = 'G235H' ##Set the right filters
pad_x = 20 #set to default values of the code so it should work but check it case it doesn't 
pad_y = 5

print('retrieving data')

obs_list = select_obs(obs_dir=datadir, fnames=fnames, enames=enames, shutters_i=shutters_i, shutters_j=shutters_j, line_wave=l0,
                      qd=qd,filt=filt,disp=disp,norm_const=f_norm,pad_y=pad_y,pad_x=pad_x) 

#Initially try to run everything to this point ------

# set location of source -- (0,0) is the centre of the shutter
rx = 0.
ry = 0.
# Main things ot chnage (initial guesses from line 94 - 100)
# set best-fit morphological params
f0 = 500.   # integrated line flux -- check units
re = 0.65    # arcsec
n_s = 1.0 # Sersic index 
pa_img = 25. # PA measured from imaging following galfit convention (E from N) in degrees
qq = 0.6 #observed axis ratios 
snr = 100 

pa = pa_img + 90. # assuming the PA is measured following galfit convention (E from N)


# uncertainties on galfit -- ignore for now
f_dq = 0.1 
f_dre = 0.1 
f_dn = 0.1 
dpa = 10. 
f_df = 0.1 
drx = 0.05
dry = 0.03

# velocity field params
vmin = -500.    # km/s maximum allowed negative velocity
vmax = 500.     # km/s maximum allowed positive velocity
smax = 200.     # km/s maximum velocity dispersion


def make_config(obs_list):

    config = {}
    config["geometry"] = {"shutter_i":[],"shutter_j":[],"quadrant":[],
                          "shutter_array":[],"source_shutter":[],
                          "psky_x":[],"psky_y":[] }
    keys = list(config["geometry"].keys())

    for i in range(len(obs_list)):
        for key in keys:
            try:
                config["geometry"][key].append(obs_list[i]["geometry"][key])
            except KeyError:
                if key=='psky_x':
                    config["geometry"][key].append(0.268)
                elif key=='psky_y':
                    config["geometry"][key].append(0.530)

    return config

def get_pix_cutout(obs_list):
    pix_cood = []
    for i in range(len(obs_list)):
        pix_cood.append(obs_list[i]["pix_cood"])
    return pix_cood


# make the PSF cube, selecting PSF at the right wavelength and slicing out unnecessary zeros
print('making PSF cube')
psf_lib = PSFLib('1x3_G140M_Q3_PSFLib.fits')
psf_lib.interp_new_wave(l0)
wsel = np.argmin(np.abs(psf_lib.psf_wave - l0))
psf_lib.slice_cube(wsel=(wsel,wsel+1),xsel=(3,-3),ysel=(3,-3))
psf_lib.slice_psfs(npix_x=9,npix_y=3)


# make the config file based on the observed data
Nobs = len(obs_list)
config = make_config(obs_list)
config["instrument"] = obs_list[0]["instrument"]
config['geometry']['shutter_array'] = ['1x3'] * Nobs 
config['geometry']['source_shutter'] = [0] * Nobs

x0 = []
b_low = []
b_up = []

# start the parameter config needed for the fitting: set fixed and variable model parameters
p_fit = {}
p_fit["line_wave"] = {"fixed":False,"prior":priors.Normal(mean=l0,sigma=2.)}
x0.append(l0)
b_low.append(l0-5)
b_up.append(l0+5)
p_fit["flux"] = {"model":"morph","fixed":False,"prior":priors.ClippedNormal(mean=f0,sigma=f_df*f0,mini=np.maximum(0,f0-3*f_df*f0),maxi=f0+5*f_df*f0)}
x0.append(f0)
b_low.append(0)
b_up.append(3*f0)

for i in range(Nobs):
    if fix_position:
        p_fit[f"x0_{i}"] = {"model":"morph","fixed":True,"value":rx}
        p_fit[f"y0_{i}"] = {"model":"morph","fixed":True,"value":ry}
    else:
        if i==0:
            p_fit[f"x0_{i}"] = {"model":"morph","fixed":False,"prior":priors.ClippedNormal(mean=rx,sigma=drx,mini=rx-3*drx,maxi=rx+3*drx)}
            p_fit[f"y0_{i}"] = {"model":"morph","fixed":False,"prior":priors.ClippedNormal(mean=ry,sigma=dry,mini=ry-3*dry,maxi=ry+3*dry)}
            x0 += [rx,ry]
            b_low += [rx-0.1,ry-0.1]
            b_up += [rx+0.1,ry+0.1]
        else:
            p_fit[f"x0_{i}"] = {"model":"morph","fixed":True,"value":rx}
            p_fit[f"y0_{i}"] = {"model":"morph","fixed":True,"value":ry}

# morphological params
p_fit["r_e"] = {"model":"morph","fixed":False,"prior":priors.ClippedNormal(mean=re,sigma=f_dre*re,mini=0.01,maxi=re+5*f_dre*re)}
x0.append(re)
b_low.append(0.5*re)
b_up.append(2*re)

if fix_nsersic: p_fit["n"] = {"model":"morph","fixed":True,"value":n_s}
else: 
    p_fit["n"] = {"model":"morph","fixed":False,"prior":priors.ClippedNormal(mean=n_s,sigma=f_dn*n_s,mini=0.5,maxi=8)}
    x0.append(n_s)
    b_low.append(0.5)
    b_up.append(8.)
p_fit["q"] = {"model":"morph","fixed":False,"prior":priors.ClippedNormal(mean=qq,sigma=f_dq*qq,mini=0.05,maxi=1.0)}
x0.append(qq)
b_low.append(np.maximum(0.1,qq-0.15))
b_up.append(np.minimum(1.0,qq+0.15))
p_fit["PA"] = {"model":"morph","fixed":False,"prior":priors.ClippedNormal(mean=pa,sigma=dpa,mini=pa-3*dpa,maxi=pa+3*dpa)}
x0.append(pa)
b_low.append(pa-15)
b_up.append(pa+15)

# velocity field params
if fix_rt: p_fit["r_t"] = {"model":"vfield","fixed":True,"value":fix_turnover}
else:
    p_fit["r_t"] = {"model":"vfield","fixed":True,"value":rturnover_from_reff}
    p_fit["frac_rt_re"] = {"model":"vfield","fixed":False,"prior":priors.Uniform(mini=0.01,maxi=1.0)} 
    x0.append(0.25)
    b_low.append(0.05)
    b_up.append(1.0)

p_fit["v_a"] = {"model":"vfield","fixed":False,"prior":priors.Uniform(mini=vmin,maxi=vmax)}
x0.append(-100.)
b_low.append(vmin)
b_up.append(vmax)
p_fit["sigma_0"] = {"model":"vfield","fixed":False,"prior":priors.Uniform(mini=0.1,maxi=smax)}
x0.append(50)
b_low.append(0)
b_up.append(smax)
p_fit["inclination"] = {"model":"vfield","fixed":True,"value":thindisk_inclination}

config["fit_params"] = p_fit  


pix_cood = get_pix_cutout(obs_list)

fmodel = FitModel(config,ThinDisk,psf_lib,fid_line_wave=l0,velrange=vmax,pix_cood=pix_cood)
if store_fmodel: 
    with open(wdir+f'fmodel.pickle','wb') as fout:
        pickle.dump(fmodel,fout)

obs_data = prep_obs(obs_list)
obs_spec,obs_unc,mask_spec = obs_data


if optimize:
    print('start optimizer')

    min_opt = run_fit_optimizer(fmodel,obs_list,'min',param_bounds=[b_low,b_up],x0=x0,disp=True, maxiter=1000,keep_feasible=True,fname_out=wdir+f'min_optimization',method='Powell',options={'ftol':1e-3})
    #diff_opt = run_fit_optimizer(fmodel,obs_list,'diff',param_bounds=[b_low,b_up],disp=True, maxiter=20,keep_feasible=True,workers=10,tol=0.01,x0=x0,fname_out=wdir+f'diff_optimization')


print(time.time()-start_time)

IndentationError: unindent does not match any outer indentation level (sampling.py, line 31)

In [ ]:
path = datadir + fnames[0]

In [ ]:
fnames[0]

In [ ]:
np.load("/u/flolac/Mocks/mock_1.npy")